# Chapter 23: State

In [1]:
:!stack install random
-- maybe its not needed actually, should be part of standard installation

In [2]:
import System.Random

In [3]:
:t mkStdGen

mkStdGen :: Int -> StdGen

In [4]:
:info StdGen

data StdGen = System.Random.StdGen {-# UNPACK #-}GHC.Int.Int32 {-# UNPACK #-}GHC.Int.Int32 	-- Defined in ‘System.Random’
instance Show StdGen -- Defined in ‘System.Random’
instance Read StdGen -- Defined in ‘System.Random’
instance RandomGen StdGen -- Defined in ‘System.Random’

In [5]:
:info RandomGen

class RandomGen g where
  next :: g -> (Int, g)
  genRange :: g -> (Int, Int)
  split :: g -> (g, g)
  {-# MINIMAL next, split #-}
  	-- Defined in ‘System.Random’
instance RandomGen StdGen -- Defined in ‘System.Random’

In [6]:
:t next

next :: forall g. RandomGen g => g -> (Int, g)

In [7]:
:info Random

class Random a where
  randomR :: RandomGen g => (a, a) -> g -> (a, g)
  random :: RandomGen g => g -> (a, g)
  randomRs :: RandomGen g => (a, a) -> g -> [a]
  randoms :: RandomGen g => g -> [a]
  randomRIO :: (a, a) -> IO a
  randomIO :: IO a
  {-# MINIMAL randomR, random #-}
  	-- Defined in ‘System.Random’
instance Random Word -- Defined in ‘System.Random’
instance Random Integer -- Defined in ‘System.Random’
instance Random Int -- Defined in ‘System.Random’
instance Random Float -- Defined in ‘System.Random’
instance Random Double -- Defined in ‘System.Random’
instance Random Char -- Defined in ‘System.Random’
instance Random Bool -- Defined in ‘System.Random’

In [8]:
:t random

random :: forall a g. (Random a, RandomGen g) => g -> (a, g)

In [9]:
mkStdGen 0

1 1

In [10]:
mkStdGen 10

11 1

In [11]:
sg = mkStdGen 0
next sg

(2147482884,40014 40692)

In [12]:
random sg :: (Float, StdGen)

(0.74242944,1601120196 1655838864)

In [13]:
import Data.Maybe

data Die
    = DieOne
    | DieTwo
    | DieThree
    | DieFour
    | DieFive
    | DieSix
    deriving (Eq, Show)

intToDie :: Int -> Maybe Die
intToDie 1 = Just DieOne
intToDie 2 = Just DieTwo  
intToDie 3 = Just DieThree
intToDie 4 = Just DieFour
intToDie 5 = Just DieFive 
intToDie 6 = Just DieSix
intToDie _ = Nothing

unsafeIntToDie :: Int -> Die
unsafeIntToDie = fromJust . intToDie

dieToInt :: Die -> Int
dieToInt DieOne = 1
dieToInt DieTwo = 2
dieToInt DieThree = 3
dieToInt DieFour = 4
dieToInt DieFive = 5
dieToInt DieSix = 6

In [14]:
:info Functor

class Functor (f :: * -> *) where
  fmap :: (a -> b) -> f a -> f b
  (<$) :: a -> f b -> f a
  {-# MINIMAL fmap #-}
  	-- Defined in ‘GHC.Base’
instance Functor IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Functor (Either a) -- Defined in ‘Data.Either’
instance Functor [] -- Defined in ‘GHC.Base’
instance Functor Maybe -- Defined in ‘GHC.Base’
instance Functor IO -- Defined in ‘GHC.Base’
instance Functor ((->) r) -- Defined in ‘GHC.Base’
instance Functor ((,) a) -- Defined in ‘GHC.Base’

In [15]:
:info Foldable

class Foldable (t :: * -> *) where
  Data.Foldable.fold :: Monoid m => t m -> m
  foldMap :: Monoid m => (a -> m) -> t a -> m
  foldr :: (a -> b -> b) -> b -> t a -> b
  Data.Foldable.foldr' :: (a -> b -> b) -> b -> t a -> b
  foldl :: (b -> a -> b) -> b -> t a -> b
  Data.Foldable.foldl' :: (b -> a -> b) -> b -> t a -> b
  foldr1 :: (a -> a -> a) -> t a -> a
  foldl1 :: (a -> a -> a) -> t a -> a
  Data.Foldable.toList :: t a -> [a]
  null :: t a -> Bool
  length :: t a -> Int
  elem :: Eq a => a -> t a -> Bool
  maximum :: Ord a => t a -> a
  minimum :: Ord a => t a -> a
  sum :: Num a => t a -> a
  product :: Num a => t a -> a
  {-# MINIMAL foldMap | foldr #-}
  	-- Defined in ‘Data.Foldable’
instance Foldable [] -- Defined in ‘Data.Foldable’
instance Foldable Maybe -- Defined in ‘Data.Foldable’
instance Foldable (Either a) -- Defined in ‘Data.Foldable’
instance Foldable ((,) a) -- Defined in ‘Data.Foldable’

In [16]:
:info Traversable

class (Functor t, Foldable t) => Traversable (t :: * -> *) where
  traverse :: Applicative f => (a -> f b) -> t a -> f (t b)
  sequenceA :: Applicative f => t (f a) -> f (t a)
  mapM :: Monad m => (a -> m b) -> t a -> m (t b)
  sequence :: Monad m => t (m a) -> m (t a)
  {-# MINIMAL traverse | sequenceA #-}
  	-- Defined in ‘Data.Traversable’
instance Traversable [] -- Defined in ‘Data.Traversable’
instance Traversable Maybe -- Defined in ‘Data.Traversable’
instance Traversable (Either a) -- Defined in ‘Data.Traversable’
instance Traversable ((,) a) -- Defined in ‘Data.Traversable’

In [17]:
import Control.Applicative (liftA3)

data Tuple3Homo a = Tuple3Homo a a a

instance Functor Tuple3Homo where
    fmap f (Tuple3Homo a b c) = Tuple3Homo (f a) (f b) (f c)
    
instance Foldable Tuple3Homo where
    foldMap f (Tuple3Homo a b c) = f a <> f b <> f c
    
instance Traversable Tuple3Homo where
    traverse f (Tuple3Homo a b c) = liftA3 Tuple3Homo (f a) (f b) (f c)
    
toTuple :: Tuple3Homo a -> (a, a, a)
toTuple (Tuple3Homo a b c) = (a, b, c)

In [18]:
rollDieThreeTimes' :: RandomGen g => g -> (Die, Die, Die)
rollDieThreeTimes' g = toTuple . fromJust . traverse intToDie $ Tuple3Homo d1 d2 d3 where
    r = randomR (1, 6)
    (d1, g1) = r g
    (d2, g2) = r g1
    (d3, _) = r g2

In [19]:
rollDieThreeTimes :: (Die, Die, Die)
rollDieThreeTimes = rollDieThreeTimes' (mkStdGen 0)
rollDieThreeTimes

(DieSix,DieSix,DieFour)

In [20]:
import Control.Monad (replicateM)
import Control.Monad.Trans.State

In [21]:
:t replicateM

replicateM :: forall (m :: * -> *) a. Applicative m => Int -> m a -> m [a]

In [22]:
:info State

type State s = StateT s Data.Functor.Identity.Identity :: * -> * 	-- Defined in ‘Control.Monad.Trans.State.Lazy’

In [23]:
:info StateT

type role StateT nominal representational nominal
newtype StateT s (m :: * -> *) a = StateT {runStateT :: s -> m (a, s)}
  	-- Defined in ‘Control.Monad.Trans.State.Lazy’
instance [safe] Functor m => Functor (StateT s m) -- Defined in ‘Control.Monad.Trans.State.Lazy’
instance [safe] (Functor m, Monad m) => Applicative (StateT s m) -- Defined in ‘Control.Monad.Trans.State.Lazy’
instance [safe] Monad m => Monad (StateT s m) -- Defined in ‘Control.Monad.Trans.State.Lazy’

In [24]:
:t state

state :: forall (m :: * -> *) s a. Monad m => (s -> (a, s)) -> StateT s m a

In [25]:
:t state (randomR (1, 6))

state (randomR (1, 6)) :: forall (m :: * -> *) a s. (Monad m, Random a, RandomGen s, Num a) => StateT s m a

In [26]:
:t intToDie <$> state (randomR (1, 6))

intToDie <$> state (randomR (1, 6)) :: forall (m :: * -> *) s. (Monad m, RandomGen s) => StateT s m (Maybe Die)

In [27]:
rollDie :: State StdGen Die
rollDie = unsafeIntToDie <$> state (randomR (1, 6))
:t rollDie

rollDie :: State StdGen Die

In [28]:
rollDieThreeTimes' :: State StdGen (Die, Die, Die)
rollDieThreeTimes' = liftA3 (,,) rollDie rollDie rollDie

In [29]:
:t evalState

evalState :: forall s a. State s a -> s -> a

In [30]:
:t runState

runState :: forall s a. State s a -> s -> (a, s)

In [31]:
evalState rollDieThreeTimes' (mkStdGen 0)

(DieSix,DieSix,DieFour)

In [32]:
:t repeat

repeat :: forall a. a -> [a]

In [33]:
infiniteDie :: State StdGen [Die]
infiniteDie = repeat <$> rollDie

In [34]:
take 6 $ evalState infiniteDie (mkStdGen 0)

[DieSix,DieSix,DieSix,DieSix,DieSix,DieSix]

In [35]:
import Control.Applicative (liftA2)

replicateM' :: Applicative m => Int -> m a -> m [a]
replicateM' cnt0 f = loop cnt0 where
    loop cnt
        | cnt <= 0  = pure []
        | otherwise = liftA2 (:) f (loop (cnt - 1))

repeatM :: Applicative m => m a -> m [a]
repeatM f = loop where
    loop = liftA2 (:) f loop

In [36]:
evalState (replicateM' 6 rollDie) (mkStdGen 0)
take 6 $ evalState (repeatM rollDie) (mkStdGen 0)

[DieSix,DieSix,DieFour,DieOne,DieFive,DieTwo]

[DieSix,DieSix,DieFour,DieOne,DieFive,DieTwo]

In [37]:
infiniteDie' :: State StdGen [Die]
infiniteDie' = repeatM rollDie

In [38]:
take 6 $ evalState infiniteDie' (mkStdGen 0)

[DieSix,DieSix,DieFour,DieOne,DieFive,DieTwo]

In [39]:
nDie :: Int -> State StdGen [Die]
nDie n = replicateM' n rollDie

In [40]:
evalState (nDie 6) (mkStdGen 0)

[DieSix,DieSix,DieFour,DieOne,DieFive,DieTwo]

In [41]:
:t scanl

scanl :: forall b a. (b -> a -> b) -> b -> [a] -> [b]

In [42]:
:t state

state :: forall (m :: * -> *) s a. Monad m => (s -> (a, s)) -> StateT s m a

In [43]:
:t runState

runState :: forall s a. State s a -> s -> (a, s)

In [44]:
:t repeatM (flip runState <$> rollDie)

repeatM (flip runState <$> rollDie) :: forall a. StateT StdGen Identity [State Die a -> (a, Die)]

In [45]:
rollsToGetN :: Int -> State StdGen (Int, [Die])
rollsToGetN n = f <$> repeatM rollDie where
    f rolls = (l, origs) where
        ints = dieToInt <$> rolls
        scans = drop 1 . scanl (+) 0 $ ints
        filtered = takeWhile (<n) scans
        l = length filtered
        origs = take l rolls

In [46]:
evalState (rollsToGetN 30) (mkStdGen 0)

(7,[DieSix,DieSix,DieFour,DieOne,DieFive,DieTwo,DieFour])

---

In [47]:
newtype Moi s a = Moi { runMoi :: s -> (a, s) }

In [48]:
instance Functor (Moi s) where
    fmap f (Moi g) = Moi h where
        h s = let (a, s') = g s in (f a, s')

In [49]:
runMoi ((+1) <$> (Moi $ \s -> (0, s))) 0

Line 1: Use tuple-section
Found:
\ s -> (0, s)
Why not:
(0,)Line 1: Use tuple-section
Found:
\ s -> (0, s)
Why not:
(0,)

(1,0)

In [50]:
instance Applicative (Moi s) where
    pure a = Moi $ \s -> (a, s)
    
    (Moi ff) <*> (Moi g) = Moi h where
        h s = (f a, s'') where
            (f, s') = ff s
            (a, s'') = g s'
    -- order of state application is defined by law:
    -- pure (.) <*> u <*> v <*> w = u <*> (v <*> w)

Line 2: Use tuple-section
Found:
\ s -> (a, s)
Why not:
(a,)Line 2: Use tuple-section
Found:
\ s -> (a, s)
Why not:
(a,)

In [51]:
:info Monad

class Applicative m => Monad (m :: * -> *) where
  (>>=) :: m a -> (a -> m b) -> m b
  (>>) :: m a -> m b -> m b
  return :: a -> m a
  fail :: String -> m a
  {-# MINIMAL (>>=) #-}
  	-- Defined in ‘GHC.Base’
instance [safe] Monad m => Monad (StateT s m) -- Defined in ‘Control.Monad.Trans.State.Lazy’
instance Monad IHaskellTH.Q -- Defined in ‘Language.Haskell.TH.Syntax’
instance Monad (Either e) -- Defined in ‘Data.Either’
instance Monad [] -- Defined in ‘GHC.Base’
instance Monad Maybe -- Defined in ‘GHC.Base’
instance Monad IO -- Defined in ‘GHC.Base’
instance Monad ((->) r) -- Defined in ‘GHC.Base’
instance Monoid a => Monad ((,) a) -- Defined in ‘GHC.Base’

In [52]:
instance Monad (Moi s) where
    (Moi f) >>= mg = Moi h where
        h s = g s' where
            (a, s') = f s
            Moi g = mg a

---

In [53]:
get' :: Moi s s
get' = Moi $ \s -> (s, s)

runMoi get' "curry is Amaze"

("curry is Amaze","curry is Amaze")

In [54]:
put' :: s -> Moi s ()
put' s = (Moi . const) ((), s)

runMoi (put' "blah") "woot"

((),"blah")

In [55]:
exec' :: Moi s a -> s -> s
exec' (Moi sa) = snd . sa

exec' (put' "wilma") "daphne"

"wilma"

In [56]:
eval' :: Moi s a -> s -> a
eval' (Moi sa) = fst . sa

eval' (put' "wilma") "daphne"

()

In [57]:
modify' :: (s -> s) -> Moi s ()
modify' f = Moi $ \s -> ((), f s)

runMoi (modify' (+1)) 0

((),1)